In [ ]:
# default_exp core

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

We can use this to say hello to our transfer learning friends.

In [ ]:
say_hello("transfer learning friends")

'Hello transfer learning friends!'

In [ ]:
from fastcore.test import *
test_eq(say_hello("transfer learning friends"), "Hello transfer learning friends!")